In [3]:
# AlphabetSoupCharity Optimization
# Import Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [8]:
# Load Data
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

# Drop unnecessary columns
application_df = application_df.drop(columns=["EIN", "NAME"])

# Bin Rare categorical variables
# Application_type
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T14', 'T25', 'T15', 'T29', 'T17']
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
# Classification
classifications_to_replace = list(application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index)
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Convert categorical data to numeric
application_df = pd.get_dummies(application_df)

# Split data into features and target
x = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

# Scale data
scaler = StandardScaler()
x_scaler = scaler.fit(x_train)
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [12]:
# Define optimized model
nn_optimized = tf.keras.models.Sequential()

# First hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=len(x_train.columns)))

# Second hidden layer (tanh)
nn_optimized.add(tf.keras.layers.Dense(units=50, activation='tanh'))

# Third hidden layer
nn_optimized.add(tf.keras.layers.Dense(units=20, activation='relu'))

#Dropout layer to prevent overfitting
nn_optimized.add(tf.keras.layers.Dropout(0.2))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile model with different optimizer
nn_optimized.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

# Train model with more epochs
fit_model = nn_optimized.fit(x_train_scaled, y_train, epochs=200)


Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7076 - loss: 0.5881
Epoch 2/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7281 - loss: 0.5594
Epoch 3/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7326 - loss: 0.5588
Epoch 4/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7268 - loss: 0.5576
Epoch 5/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7332 - loss: 0.5522
Epoch 6/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7330 - loss: 0.5553
Epoch 7/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7336 - loss: 0.5565
Epoch 8/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7336 - loss: 0.5545
Epoch 9/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7314 - loss: 0.5522
Epoch 10/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7331 - loss: 0.5547
Epoch 11/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7276 - loss: 0.5555
Epoch 12/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step

In [13]:
# Check the model using the test data
model_loss, model_accuracy = nn_optimized.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 2ms/step - accuracy: 0.7306 - loss: 0.5620
Loss: 0.5620313286781311, Accuracy: 0.7306122183799744


In [15]:
# Export optimized model to hdf5 file
nn_optimized.save("AlphabetSoupCharity_Optimization.keras")